# Model MLCLSP

(ang. Multi-level Capacitated Lot-sizing Problem, MLCLSP)

### Parametry:

$\mathcal{T}$ -- zbiór okresów czasu (zmian, dni lub tygodni),

$\mathcal{N}$ -- zbiór wyrobów,

$\mathcal{M}$ -- zbiór maszyn,

$a_{ij}$ -- liczba wyrobów $i$ potrzebnych do wyprodukowania jednego wyrobu $j$,

$p_{mi}$ -- czas potrzebny na wyprodukowanie wyrobu $i$ na maszynie $m$,

$c_i$ -- koszt przezbrojenia na wykonanie wyrobu $i$,

$E_{it}$ -- zewnętrzne zapotrzebowanie na wyrób $i$ w okresie $t$,

$G$ -- dowolnie duża liczba,

$I_{i0}$ -- zapas początkowy wyrobu $i$,

$h_i$ -- jednostkowy koszt utrzymywania zapasów wyrobu $i$,

$L_{mt}$ -- dostępny czas pracy maszyny $m$ w okresie $t$,

$s_{mi}$ -- czas potrzebny na przezbrojenie maszyny $m$ na produkcję wyrobu $i$,

$l_i$ -- czas realizacji = 0,

### Zmienne:

$I_{it}$ -- zapas wyrobu $i$ na koniec okresu $t$,

$x_{it}$ -- wielkość produkcji wyrobu $i$ w okresie $t$,

$y_{it} = 1$,  jeżeli  wyrób $i$ jest produkowany w okresie $t$, $0$ w przeciwnym razie.

In [1]:
import numpy as np
import pandas as pd

---
## Zaczytanie danych wejściowych z pliku excel

In [2]:
# nazwy kolejnych wyrobów, maszyn i okresów w pliku excel muszą być kolejnymi liczbami naturalnymi zaczynającymi się o cyfry 0
wejscie_wyroby = pd.read_excel('szablony/MLCLSP_wejscie.xlsx', sheet_name='WYROBY')
wejscie_okresy = pd.read_excel('szablony/MLCLSP_wejscie.xlsx', sheet_name='ZAPOTRZEBOWANIE')
wejscie_czas_produkcji = pd.read_excel('szablony/MLCLSP_wejscie.xlsx', sheet_name='CZAS_PRODUKCJI')
wejscie_czas_przezbrojenia = pd.read_excel('szablony/MLCLSP_wejscie.xlsx', sheet_name='CZAS_PRZEZBROJENIA')
wejscie_dostepnosc_maszyn = pd.read_excel('szablony/MLCLSP_wejscie.xlsx', sheet_name='DOSTEPNOSC_MASZYN')
bom = pd.read_excel('szablony/MLCLSP_wejscie.xlsx', sheet_name='BOM')

print(wejscie_wyroby)
print(wejscie_okresy)
print(wejscie_czas_produkcji)
print(wejscie_czas_przezbrojenia)
print(wejscie_dostepnosc_maszyn)
print(bom)

   wyrob  hi  ci  Ii0  li
0      0   3   5    0   0
1      1   2   5    0   0
2      2   2   5    0   0
3      3   1   5    0   0
   wyrob/okres  1  2
0            0  3  0
1            1  0  2
2            2  0  0
3            3  0  0
   pmi    0    1    2
0    0  0.1  0.0  0.0
1    1  0.0  0.4  0.0
2    2  0.0  0.0  0.1
3    3  0.0  0.0  0.1
   smi     0     1     2
0    0  0.05  0.00  0.00
1    1  0.00  0.05  0.00
2    2  0.00  0.00  0.05
3    3  0.00  0.00  0.05
   Lmt  1  2
0    0  1  1
1    1  1  1
2    2  1  1
   j/i  0  1  2  3
0    0  0  0  0  0
1    1  0  0  0  0
2    2  1  0  0  0
3    3  0  1  1  0


---
## Wyciągnięcie danych z zaczytanych tabel

In [3]:
zbior_wyrobow = wejscie_wyroby.loc[:, 'wyrob'].values
zbior_okresow = wejscie_okresy.iloc[:, 1:].columns.values
zbior_maszyn = wejscie_czas_produkcji.iloc[:, 1:].columns.values
zbior_okresow_wraz_z_poczatkowym = np.append(0, zbior_okresow)

print('Zbiór wyrobów: ', zbior_wyrobow)
print('Zbiór okresów: ', zbior_okresow)
print('Zbiór maszyn: ', zbior_maszyn)
print('Zbiór okresów z okresem początkowym: ', zbior_okresow_wraz_z_poczatkowym)

Zbiór wyrobów:  [0 1 2 3]
Zbiór okresów:  [1 2]
Zbiór maszyn:  [0 1 2]
Zbiór okresów z okresem początkowym:  [0 1 2]


In [4]:
h = wejscie_wyroby.loc[:, 'hi'].values
c = wejscie_wyroby.loc[:, 'ci'].values
l = wejscie_wyroby.loc[:, 'li'].values
I_0 = wejscie_wyroby.loc[:, 'Ii0'].values

print('Jednostkowy koszt utrzymania wyrobu i: ', h)
print('Koszt przezbrojenia na wykonanie wyrobu i: ', c)
print('Czas realizacji dla wyrobu i: ', l)
print('Zapasy początkowe: ', I_0)

Jednostkowy koszt utrzymania wyrobu i:  [3 2 2 1]
Koszt przezbrojenia na wykonanie wyrobu i:  [5 5 5 5]
Czas realizacji dla wyrobu i:  [0 0 0 0]
Zapasy początkowe:  [0 0 0 0]


In [5]:
a = bom.set_index('j/i').values
E = wejscie_okresy.values
p = wejscie_czas_produkcji.set_index('pmi').values
s = wejscie_czas_przezbrojenia.set_index('smi').values
L = wejscie_dostepnosc_maszyn.values
G = 1000000

print('Macierz powiązań między produktami: \n', a)
print('Macierz z zewnętrznym zapotrzebowaniem na produkt i w okresie t (pierwsza kolumna to odpowiednik produktu): \n',
      E)
print('Czas produkcji wyrobu i ma maszynie m: \n', p)
print('Czas przezbrojenia maszyny m na produkcję wyrobu i: \n', s)
print('Dostępnośc maszyny m w okresie t (pierwsza kolumna to odpowiednik maszyny): \n', L)

Macierz powiązań między produktami: 
 [[0 0 0 0]
 [0 0 0 0]
 [1 0 0 0]
 [0 1 1 0]]
Macierz z zewnętrznym zapotrzebowaniem na produkt i w okresie t (pierwsza kolumna to odpowiednik produktu): 
 [[0 3 0]
 [1 0 2]
 [2 0 0]
 [3 0 0]]
Czas produkcji wyrobu i ma maszynie m: 
 [[0.1 0.  0. ]
 [0.  0.4 0. ]
 [0.  0.  0.1]
 [0.  0.  0.1]]
Czas przezbrojenia maszyny m na produkcję wyrobu i: 
 [[0.05 0.   0.  ]
 [0.   0.05 0.  ]
 [0.   0.   0.05]
 [0.   0.   0.05]]
Dostępnośc maszyny m w okresie t (pierwsza kolumna to odpowiednik maszyny): 
 [[0 1 1]
 [1 1 1]
 [2 1 1]]


---
## Sprawdzenie poprawności wpisanych danych dla czasu produkcji oraz czasu przezbrojenia

### Funkcje pomocnicze

In [6]:
def sprawdz_poprawnosc_dla(tabela, nazwa_arkusza):
    for kolumna in tabela:
        if np.count_nonzero(kolumna) != 1:
            raise Exception(
                'Każdy wyrób może być produkowany tylko na jednej maszynie. Należy poprawić dane w arkuszu ' + nazwa_arkusza)


In [7]:
sprawdz_poprawnosc_dla(p, 'CZAS_PRODUKCJI')

---
## Utworzenie modelu Gurobi oraz zmiennych

$I_{it} >= 0,
x_{it} >= 0,
y_{it} \in {0,1} \qquad i \in \mathcal{N}, t \in \mathcal{T},$


In [8]:
import gurobipy as gb

model = gb.Model('MLCLSP')

# Deklaracje zmiennych (decyzyjnych)
x = {}
y = {}
I = {}
for i in zbior_wyrobow:
    for t in zbior_okresow:
        x[i, t] = model.addVar(vtype=gb.GRB.INTEGER, lb=0, name='x(' + str(i) + ',' + str(t) + ')')
        y[i, t] = model.addVar(vtype=gb.GRB.BINARY, name='y(' + str(i) + ',' + str(t) + ')')

for i in zbior_wyrobow:
    for t in zbior_okresow_wraz_z_poczatkowym:
        I[i, t] = model.addVar(vtype=gb.GRB.INTEGER, lb=0, name='I(' + str(i) + ',' + str(t) + ')')

---
## Funkcja celu
$
\min \displaystyle\sum_{i \in \mathcal{N}} \sum_{t \in \mathcal{T}}( c_i y_{it} + h_i I_{it} )
$

In [9]:
model.setObjective(gb.quicksum(h[i] * I[i, t] + c[i] * y[i, t] for i in zbior_wyrobow for t in zbior_okresow))

---
## Ograniczenia
$I_{i0} = I^0_i$

In [10]:
model.addConstrs((I[i, 0] == I_0[i] for i in zbior_wyrobow), name='zapas-poczatkowy')
model.update()

$I_{it} = I_{i(t-1)} + x_{i(t-l_i)} - \sum_{j \in \mathcal{N}}( a_{ij} * x_{jt} ) - E_{it} \qquad i \in \mathcal{N}, t \in \mathcal{T},$

In [11]:
for i in zbior_wyrobow:
    for t in zbior_okresow:
        model.addConstr(
            I[i, t] == I[i, t - 1] + x[i, t - l[i]] - sum(a[i, j] * x[j, t] for j in zbior_wyrobow) - E[i, t],
            name='bilans_zapasow(' + str(i) + ',' + str(t) + ')'
        )

$ \sum_{i \in \mathcal{N}}( p_{mi} * x_{it} + s_{mi} * y_{it}) <= L_{mt} \qquad m \in \mathcal{M}, t \in \mathcal{T},$

In [12]:
for m in zbior_maszyn:
    for t in zbior_okresow:
        model.addConstr(sum(p[i, m] * x[i, t] + s[i, m] * y[i, t] for i in zbior_wyrobow) <= L[m, t],
                        name='czas_pracy_maszyny(' + str(m) + ',' + str(t) + ')'
                        )

$x_{it} - G * y_{it} <= 0 \qquad i \in \mathcal{N}, t \in \mathcal{T},$

In [13]:
for i in zbior_wyrobow:
    for t in zbior_okresow:
        model.addConstr(
            x[i, t] - G * y[i, t] <= 0,
            name='produkcja_wyrobu(' + str(i) + ',' + str(t) + ')'
        )

---
## Optymalizacja

In [14]:
# Aktualizacja całego modelu, obowiązkowa przed optymalizacją
model.update()

# Optymalizacja
model.optimize()

# Zapisanie modelu do pliku
model.write('szablony/MLCLSP_model.lp')
model.write('szablony/MLCLSP_model.mps')


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 28 columns and 66 nonzeros
Model fingerprint: 0x06bd9458
Variable types: 0 continuous, 28 integer (8 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+06]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 24.0000000
Presolve removed 22 rows and 22 columns
Presolve time: 0.06s
Presolved: 4 rows, 6 columns, 9 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)

Root relaxation: objective 2.200000e+01, 2 iterations, 0.04 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      22.0000

---
## Wydruk modelu

### Funkcje pomocnicze

In [15]:

def zamien_kropkw_na_przecinek(x):
    return (str(x).replace('.', ','))


def wypisz_parametry_modelu(writer):
    writer.writerow(['Zmienna', 'Wyrób i', 'Okres', 'Maszyna', 'Wyrób j', 'Wartość'])

    writer.writerow(['Nazwa modelu', '', '', '', '', model.ModelName])
    writer.writerow(['Wersja solvera', '', '', '', '', 'Gurobi {}.{}.{}'.format(*gb.gurobi.version())])
    writer.writerow(['Data rozwiązania', '', '', '', '', time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())])
    writer.writerow(['Liczba ograniczeń', '', '', '', '', model.NumConstrs])
    writer.writerow(['Liczba zmiennych', '', '', '', '', model.NumVars])
    writer.writerow(['Liczba zmiennych binarnych', '', '', '', '', model.NumBinVars])
    writer.writerow(['Czas rozwiązywania', '', '', '', '', zamien_kropkw_na_przecinek(round(model.Runtime, 1))])
    writer.writerow(['Liczba iteracji', '', '', '', '', int(model.IterCount)])
    writer.writerow(['Rozwiązanie', '', '', '', '', zamien_kropkw_na_przecinek(round(model.ObjBound, 1))])

    writer.writerow(['n', '', '', '', '', zbior_wyrobow.size])
    writer.writerow(['T', '', '', '', '', zbior_okresow.size])

    # dane wejsciowe
    for wyrob in zbior_wyrobow:
        writer.writerow(['h', wyrob, '', '', '', zamien_kropkw_na_przecinek(h[wyrob])])
        writer.writerow(['c', wyrob, '', '', '', zamien_kropkw_na_przecinek(c[wyrob])])
        writer.writerow(['l', wyrob, '', '', '', zamien_kropkw_na_przecinek(l[wyrob])])
        writer.writerow(['I_0', '', '0', '', '', zamien_kropkw_na_przecinek(I_0[wyrob])])

    for wyrob_i in zbior_wyrobow:
        for wyrob_j in zbior_wyrobow:
            writer.writerow(['a', wyrob_i, '', '', wyrob_j, zamien_kropkw_na_przecinek(a[wyrob_i, wyrob_j])])

    for wyrob in zbior_wyrobow:
        for okres in zbior_okresow:
            writer.writerow(['E', wyrob, okres, '', '', zamien_kropkw_na_przecinek(E[wyrob, okres])])

    for wyrob in zbior_wyrobow:
        for maszyna in zbior_maszyn:
            writer.writerow(['pmi', wyrob, '', maszyna, '', zamien_kropkw_na_przecinek(p[wyrob, maszyna])])
            writer.writerow(['smi', wyrob, '', maszyna, '', zamien_kropkw_na_przecinek(s[wyrob, maszyna])])

    for okres in zbior_okresow:
        for maszyna in zbior_maszyn:
            writer.writerow(['L', '', okres, maszyna, '', zamien_kropkw_na_przecinek(L[maszyna, okres])])

### Wydruk

In [16]:
import csv, time

plik_z_rozwiazaniem = 'szablony/MLCLSP_wyniki.csv'

with open(plik_z_rozwiazaniem, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';', quoting=csv.QUOTE_NONE)
    # Czy rozwiazanie jest wykonalne
    if model.status == gb.GRB.INFEASIBLE:
        # rowiązanie niewykonalne
        model.computeIIS()
        wypisz_parametry_modelu(writer)
        writer.writerow(['Przez podane zmienne model jest niewykonalny:'])
        for c in model.getConstrs():
            if c.IISConstr:
                writer.writerow(['Parametr', c.constrName])
    else:
        # rozwiązanie wykonalne
        wypisz_parametry_modelu(writer)

        for wyrob in zbior_wyrobow:
            for okres in zbior_okresow:
                writer.writerow(['y', wyrob, okres, '', '', zamien_kropkw_na_przecinek(round(y[wyrob, okres].x, 1))])
                writer.writerow(['x', wyrob, okres, '', '', zamien_kropkw_na_przecinek(round(x[wyrob, okres].x, 1))])
                writer.writerow(['I', wyrob, okres, '', '', zamien_kropkw_na_przecinek(round(I[wyrob, okres].x, 1))])